In [1]:
import os
import numpy as np 
import pandas as pd
from utils.utils import *
from utils.rf_gaussians import gaussian_2d_curve, gaussian_2d_curve_pol

In [23]:
columns = ["x0", "y0", "sigma", "slope", "intercept"]
bounds = params['bounds']

for i, sub in enumerate(subj_list):

    print(f'Fixing for {sub} --------')
    betas_file = os.path.join(betas_dir , f'{sub}_betas_list_nativesurface_train.npy') # could parametertize the targetsurface but eh
    betas = np.load(betas_file, allow_pickle=False).astype(np.float32).T
    n_betas, n_voxels = betas.shape

    betas_test_file = os.path.join(betas_dir, f'{sub}_betas_list_nativesurface_test.npy')
    print(f'LOADING TEST BETAS FOR SUBJ0{i+1}')
    betas_test = np.load(betas_test_file, allow_pickle=False).astype(np.float32).T

    train_test_mask_file = os.path.join(betas_dir, f'{sub}_betas_list_nativesurface_train_test_mask.npy')
    print(f'LOADING TRAIN TEST MASK FOR SUBJ0{i+1}')
    train_test_mask = np.load(train_test_mask_file, allow_pickle=False).T.astype(bool)

    
    for j, roi in enumerate(rois.keys()):
        print(f"Fixing fits for {roi}")
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')
        fits_roi = np.load(fits_path, allow_pickle=True)
        fits_roi = fits_roi[:, :5]
        fits_roi = pd.DataFrame(fits_roi, columns=columns)

        mds_file = os.path.join(mds_dir, sub, f'{sub}_{roi}_MDS_rotated_VO-1_train.npy')  # we only care about rotate
        mds = np.load(mds_file, allow_pickle=True).astype(np.float32).T


        pred_activity = fits_roi.apply(gaus_roi, axis = 1)
        pred_activity = np.array([np.array(x) for x in pred_activity]).T
        roi_res = np.sum((pred_activity - betas)**2, axis=0)
        roi_tot = sum((betas- np.tile(betas.mean(axis=0), (n_betas, 1)))**2).T


        mds_test = mds[:, train_test_mask]
        pred_activity_test = fits_roi.apply(gaus_roi_test, axis=1)
        pred_activity_test = np.array([np.array(x) for x in pred_activity_test]).T
        roi_res_test = np.sum((pred_activity_test - betas_test)**2, axis=0)
        roi_rot_test = sum((betas_test - np.tile(betas_test.mean(axis=0), (sum(train_test_mask), 1)))**2).T
        fits_roi["test_var_explained"] = 1 - roi_res_test / roi_rot_test

        fits_roi["var_explained"] = 1 - roi_res / roi_tot
        fits_roi["mds_ecc"] = (fits_roi.x0 ** 2 + fits_roi.y0[1] ** 2) ** (1/2)
        fits_roi["mds_ang"] = np.arctan2(fits_roi.x0/bounds[1][0], fits_roi.y0/bounds[1][1])

        np.save(fits_path, fits_roi)
        print(f'file for {roi} has been fixed and saved ')




def gaus_roi(fits):
    return gaussian_2d_curve_pol(mds, *fits)

def gaus_roi_test(fits):
    return gaussian_2d_curve_pol(mds_test, *fits)

Fixing for subj01 --------
LOADING TEST BETAS FOR SUBJ01
LOADING TRAIN TEST MASK FOR SUBJ01
Fixing fits for V1
file for V1 has been fixed and saved 
Fixing fits for V2
file for V2 has been fixed and saved 
Fixing fits for V3
file for V3 has been fixed and saved 
Fixing fits for hV4
file for hV4 has been fixed and saved 
Fixing fits for VO-1
file for VO-1 has been fixed and saved 
Fixing fits for VO-2
file for VO-2 has been fixed and saved 
Fixing fits for PHC-1
file for PHC-1 has been fixed and saved 
Fixing fits for PHC-2
file for PHC-2 has been fixed and saved 
Fixing fits for LO-1
file for LO-1 has been fixed and saved 
Fixing fits for LO-2
file for LO-2 has been fixed and saved 
Fixing fits for TO-1
file for TO-1 has been fixed and saved 
Fixing fits for TO-2
file for TO-2 has been fixed and saved 
Fixing for subj02 --------
LOADING TEST BETAS FOR SUBJ02
LOADING TRAIN TEST MASK FOR SUBJ02
Fixing fits for V1
file for V1 has been fixed and saved 
Fixing fits for V2
file for V2 has be